# Cepheids

Cepheids are a type of variable star that pulsate radially, varying in both diameter and temperature, which produces changes in brightness with a well-defined stable period and amplitude. These stars are significant in the field of astronomy for several reasons:

1. **Standard Candles**: Cepheids have a well-established relationship between their luminosity and pulsation period, known as the Leavitt law or period-luminosity relation. This makes them excellent standard candles for measuring astronomical distances.

2. **Distance Measurement**: By observing the period of a Cepheid's brightness variations, astronomers can determine its absolute magnitude. Comparing this with the apparent magnitude allows for the calculation of the distance to the star, which is crucial for mapping the scale of the universe.

3. **Galactic and Extragalactic Studies**: Cepheids are used to measure distances within our galaxy and to nearby galaxies. This helps in understanding the structure and scale of the Milky Way and the local group of galaxies.

4. **Historical Importance**: The discovery of Cepheids in the Andromeda galaxy by Edwin Hubble was pivotal in establishing that the universe is expanding, leading to the formulation of the Big Bang theory.

Cepheids continue to be a vital tool in modern astrophysics, aiding in the calibration of other distance measurement methods and contributing to our understanding of the cosmos.

# Part 1

The data in ```cepheids.csv``` is organised in two columns: the period of the cepheid variable and its absolute magnitude calculated from mean luminosity.

These two values are actually related by the Leavitt law:
$$M = a \cdot \log_{10}{P} + b$$

First, we find the constants $a$ and $b$ in this relation to fit the given data.

In [ ]:
import numpy as np
import math

#reading csv file and storing its data
with open('cepheids.csv', 'r') as f:
    lines = f.read().split('\n') # split each line
del lines[0]

data_cepheids=[]
for line in lines:
    data_cepheids.append(line.split(','))
del data_cepheids[len(data_cepheids)-1]

x,y=[],[]
for i in data_cepheids:
    y.append(float(i[1]))
    x.append(math.log10(float(i[0])))

#creating arrays for abs magnitudes and log of period
y=np.array(y)
x=np.array(x)

y_avg=np.average(y)
x_avg=np.average(x)

#finding std deviation for each data value
y_dev=np.array(y-y_avg)
x_dev=np.array(x-x_avg)

s1=0.0
s2=0.0
for i in range(0,len(y_dev)):
    s1=s1+ (x_dev[i])*(y_dev[i])
    s2=s2+ (x_dev[i])**2

#finding the slope and intercept for the best fit line
a=s1/s2
print("The value of a is",a)
b=y_avg-a*x_avg
print("The value of b",b)

Next, we plot the data along with the fitted line. This serves as a visual verification that we have found the correct constants.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(x, y, 'ro')
x1=np.linspace(x[0],max(x),2)
y1=a*x1+b
plt.plot(x1, y1, '-')
plt.gca().invert_yaxis()
plt.xlabel("log(Period)")
plt.ylabel("Magnitude")
plt.title("Graphical Representation of Leavitt's Law")
plt.show()

# Part 2

Now we are ready to use the result we have obtained to solve the problem.

Given in the ```curves.csv``` file is the data of the light curves of many cepheids in the line of sight of a galaxy. The data contains three columns: ID(Unique for every cepheid), JD (the julian date of observation) and the apparent magnitude observed.

First we need to find the period of each cepheid.

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
#Reference: https://docs.astropy.org/en/stable/timeseries/lombscargle.html
#Reference: https://web.iucaa.in/~dipankar/CMA2020/MF_files/VanderPlas_1703.09824.pdf

#defining function to find period of a cepheid using Lomb Scargle Method
def find_period(x):
    jd, m = [], []
    for i in x:
        jd.append(i[1])
        m.append(i[2])
    jd=np.array(jd)
    m=np.array(m)

    # Compute the Lomb-Scargle periodogram
    frequency, power = LombScargle(jd, m).autopower()

    # Identify the best period
    best_frequency = frequency[power.argmax()]
    best_period = 1 / best_frequency

    return best_period

#returns mean apparent magnitude
def mean_appmag(x):
    m=[]
    for i in x:
        m.append(i[2])
    m=np.array(m)
    return np.mean(m)

#reading file
with open('curves.csv', 'r') as f:
    lines = f.read().split('\n') # split each line
del lines[0]

data_curves=[]
for line in lines:
    data_curves.append(line.split(','))
del data_curves[len(data_curves)-1]

cepheid_id, cepheid_period, cepheid_appmag=[], [], []
j=0
first, last= [], []

for i in data_curves:
    i[0]=int(i[0])
    i[1]=float(i[1])
    i[2]=float(i[2])
    if i[0] not in cepheid_id:
        cepheid_id.append(i[0])
        #storing start and end indices for a particular cepheid id
        first.append(j)
        last.append(j-1)
    j=j+1
last.append(291749)
del last[0]

print("{:<15}{:<8}".format("ID", "Period (days)"))
print("-"*33)
for i in range(len(cepheid_id)):
    x=data_curves[first[i]:last[i]+1]
    cepheid_period.append(find_period(x))
    cepheid_appmag.append(mean_appmag(x))
    print(f"{cepheid_id[i]:<15}{cepheid_period[i]:<8}")

Next, using the calculated periods, we find the absolute magnitude of these stars using the relation you found earlier.

In [ ]:
# Values of a ad b calculated in part 1
a=-2.21641005981804
b=-1.6808674146142557

# USing leavitt law to calculate absolute magnitude
def abs_mag(p):
    return a*math.log10(p)+b

cepheid_absmag=[]
print("{:<15}{:<8}".format("ID", "Absolute Magnitude"))
print("-"*33)
for i in range(len(cepheid_id)):
    x=data_curves[first[i]:last[i]+1]
    cepheid_absmag.append(abs_mag(cepheid_period[i]))
    print(f"{cepheid_id[i]:<15}{cepheid_absmag[i]:<8}")

Then, using the apparent magnitude data, we find the distance to these cepheid variables.

In [ ]:
#defining function to return distance of cepheid from us
def calc_distance(m,M):
    d=pow(10,(m-M)/5.0+1)
    return d

cepheid_distance=[]
print("{:<15}{:<8}".format("ID", "Distance (in parsecs)"))
print("-"*33)
for i in range(len(cepheid_id)):
    cepheid_distance.append(calc_distance(cepheid_appmag[i], cepheid_absmag[i]))
    print(f"{cepheid_id[i]:<15}{cepheid_distance[i]:<8}")

Finally, we find the distance to the galaxy being observed.

In [ ]:
print("The distance of the galaxy from us is around", round(np.mean(cepheid_distance)),"parsecs.")

# Part 3

Cepheid variable stars are known for their periodic changes in luminosity due to their pulsations. These stars exhibit a well-defined relationship between their pulsation period and intrinsic luminosity, known as the period-luminosity relation. But, what's the mechanism driving these pulsations? 

##### The answer lies within a phenomenon called **κ–mechanism**.
Cepheids change brightness due to the thermal instability caused by κ–mechanism.

##### Opacity is the measure of impenetrability to electromagnetic or other kinds of radiation. The kappa mechanism occurs when opacity in a star increases with temperature rather than decreasing. For example, doubly ionized helium has higher opacity than singly ionized helium.

##### The main gas thought to be involved in the process is helium. The kappa mechanism occurs in two phases:

##### A) Compression Phase:
 When the star contracts, it temperature and density in the ionisation zone increases. This rise in temperature leads to a point where double ionisation occurs spontaneously and is sustained throughout the layer, eventually leading to higher opacity. High opacity traps the radiation within the ionization zone, causing it to heat up further and store energy. The increased thermal pressure pushes the outer layers outward, initiating the expansion phase. Cepheid variables become dimmest during the part of the cycle when the helium is doubly ionized.

##### B) Expansion Phase:
  As it expands, it cools, but remains ionised until another threshold is reached at which point double ionization cannot be sustained. Thereafter, helium ions recombine; the layer becomes singly ionized hence more transparent (i.e. opacity decreases), which allows radiation to escape. The reduced pressure slows and ultimately stops the expansion. Contraction begins due to the star's gravitational attraction.
  
The thermal instability arises because the energy trapping (during compression) and release (during expansion) are out of phase with the star's motion. This ensures that the pulsations are self-sustaining, as the trapped energy drives the outward motion and the energy release facilitates contraction.

